In [2]:
import sys
#!{sys.executable} -m pip install torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from models.features import Number, Category, Sequence, Features
from models.transformers.column import (StandardScaler, CategoryEncoder, SequenceEncoder)
from models.pytorch.data import Dataset
from models.pytorch import WideDeep, DeepFM, DNN, DIN, AttentionGroup
from functions import fit, predict, create_dataloader_fn



train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.movieId = train_df.movieId.astype(str)
test_df.movieId = test_df.movieId.astype(str)

train_df.histHighRatedMovieIds = train_df.histHighRatedMovieIds.astype(str)
test_df.histHighRatedMovieIds = test_df.histHighRatedMovieIds.astype(str)

train_df.negHistMovieIds = train_df.negHistMovieIds.astype(str)
test_df.negHistMovieIds = test_df.negHistMovieIds.astype(str)

In [3]:
number_features = [
    Number('age', StandardScaler())
]

category_features = [
    Category('gender', CategoryEncoder(min_cnt=1)),
    Category('movieId', CategoryEncoder(min_cnt=1)),
    Category('occupation', CategoryEncoder(min_cnt=1)),
    Category('zipCode', CategoryEncoder(min_cnt=1))
]

sequence_features = [
    Sequence('genres', SequenceEncoder(sep='|', min_cnt=1)),
    Sequence('histHighRatedMovieIds', SequenceEncoder(sep='|', min_cnt=1)),
    Sequence('negHistMovieIds', SequenceEncoder(sep='|', min_cnt=1))
]

features, train_loader, valid_loader = create_dataloader_fn(
    number_features, category_features, sequence_features, 64, train_df, 'label', test_df, 4)

def evaluation(df, dataloader):
    preds = predict(model, dataloader)
    return roc_auc_score(df['label'], preds.ravel())


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

din_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1)]

gru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='GRU')]

aigru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AIGRU')]

agru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AGRU')]

augru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

augru_attention_groups_with_neg = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds', 'neg_hist': 'negHistMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

models = [
    DNN(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    WideDeep(
        features,
        wide_features=['genres', 'movieId'],
        deep_features=['age', 'gender', 'movieId', 'occupation',
         'zipCode', 'histHighRatedMovieIds'],
        cross_features=[('movieId', 'histHighRatedMovieIds')],
        num_classes=2, embedding_size=16, hidden_layers=(32, 16),
        final_activation='sigmoid', dropout=0.3),
    DeepFM(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIN(features, din_attention_groups, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3)
]

scores = []
for model in models:
    print(model)
    loss_func = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    fit(10, model, loss_func, optimizer,
        train_loader, valid_loader, notebook=True, auxiliary_loss_rate=0.1)
    scores.append(evaluation(test_df, valid_loader))


DNN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2849, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=113, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3, inplace=False)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

GPU is available, transfer model to GPU.
WideDeep(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2849, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): SumPooling()
  (pooling:histHighRatedMovieIds): SumPooling()
  (pooling:negHistMovieIds): SumPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=81, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3, inplace=False)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, mo

GPU is available, transfer model to GPU.
DeepFM(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2849, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (embedding:age): Linear(in_features=1, out_features=16, bias=False)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (linear): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=1, out_features=1, bias=True)
      (batchnorm0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (fm): FM()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=129, out_features=32,

GPU is available, transfer model to GPU.
DIN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:histHighRatedMovieIds): Embedding(2849, 16, padding_idx=0)
  (embedding:negHistMovieIds): Embedding(3886, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Attention(
    (mlp): MLP(
      (_sequential): Sequential(
        (dense0): Linear(in_features=64, out_features=16, bias=True)
        (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation0): PReLU(num_parameters=1)
        (dropout0): Dropout(p=0.1, inplace=False)
        (dense1): Linear(in_features=16, out_features=8, bias=True)
        (batchnorm1): BatchNorm

GPU is available, transfer model to GPU.


In [5]:
print(scores)

[0.5770925499353129, 0.5855605692467893, 0.5559490076046827, 0.5674368117131047]


In [6]:
# share embeddings

word2idx = features.category_features[1].column_flow.transformers[0].word2idx
idx2word = features.category_features[1].column_flow.transformers[0].idx2word

number_features = [
    Number('age', StandardScaler())
]

category_features = [
    Category('gender', CategoryEncoder(min_cnt=1)),
    Category('movieId',
             CategoryEncoder(word2idx=word2idx, idx2word=idx2word),
             embedding_name='movieId'),
    Category('occupation', CategoryEncoder(min_cnt=1)),
    Category('zipCode', CategoryEncoder(min_cnt=1))
]

sequence_features = [
    Sequence('genres', SequenceEncoder(sep='|', min_cnt=1)),
    Sequence('histHighRatedMovieIds',
             SequenceEncoder(sep='|', word2idx=word2idx, idx2word=idx2word),
             embedding_name='movieId'),
    Sequence('negHistMovieIds',
             SequenceEncoder(sep='|', word2idx=word2idx, idx2word=idx2word),
             embedding_name='movieId')
]

features, train_loader, valid_loader = create_dataloader_fn(
    number_features, category_features, sequence_features, 64, train_df, 'label', test_df, 4)


In [7]:
din_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1)]

gru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='GRU')]

aigru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AIGRU')]

agru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AGRU')]

augru_attention_groups = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

augru_attention_groups_with_neg = [
    AttentionGroup(
        name='group1',
        pairs=[{'ad': 'movieId', 'pos_hist': 'histHighRatedMovieIds', 'neg_hist': 'negHistMovieIds'}],
        hidden_layers=[16, 8], att_dropout=0.1, gru_type='AUGRU')]

models = [
    DNN(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    WideDeep(
        features,
        wide_features=['genres', 'movieId'],
        deep_features=['age', 'gender', 'movieId', 'occupation',
         'zipCode', 'histHighRatedMovieIds'],
        cross_features=[('movieId', 'histHighRatedMovieIds')],
        num_classes=2, embedding_size=16, hidden_layers=(32, 16),
        final_activation='sigmoid', dropout=0.3),
    DeepFM(features, 2, 16, (32, 16), final_activation='sigmoid', dropout=0.3),
    DIN(features, din_attention_groups, 2, 16, (32, 16),
        final_activation='sigmoid', dropout=0.3)
]

In [8]:
scores = []
for model in models:
    print(model)
    loss_func = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    fit(10, model, loss_func, optimizer,
        train_loader, valid_loader, notebook=True, auxiliary_loss_rate=0.1)
    scores.append(evaluation(test_df, valid_loader))

DNN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=113, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3, inplace=False)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=False)
    )
  )
  (final_layer): Linear(in_features=16, out_fe

GPU is available, transfer model to GPU.
WideDeep(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): SumPooling()
  (pooling:histHighRatedMovieIds): SumPooling()
  (pooling:negHistMovieIds): SumPooling()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=81, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU()
      (dropout0): Dropout(p=0.3, inplace=False)
      (dense1): Linear(in_features=32, out_features=16, bias=True)
      (batchnorm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=False)
    )
  )
 

GPU is available, transfer model to GPU.
DeepFM(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (embedding:age): Linear(in_features=1, out_features=16, bias=False)
  (pooling:genres): MaxPooling()
  (pooling:histHighRatedMovieIds): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (linear): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=1, out_features=1, bias=True)
      (batchnorm0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (fm): FM()
  (mlp): MLP(
    (_sequential): Sequential(
      (dense0): Linear(in_features=129, out_features=32, bias=True)
      (batchnorm0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation0): ReLU(

GPU is available, transfer model to GPU.
DIN(
  (embedding:gender): Embedding(4, 16, padding_idx=0)
  (embedding:movieId): Embedding(1610, 16, padding_idx=0)
  (embedding:occupation): Embedding(23, 16, padding_idx=0)
  (embedding:zipCode): Embedding(2676, 16, padding_idx=0)
  (embedding:genres): Embedding(20, 16, padding_idx=0)
  (pooling:genres): MaxPooling()
  (pooling:negHistMovieIds): MaxPooling()
  (attention_pooling:group1): Attention(
    (mlp): MLP(
      (_sequential): Sequential(
        (dense0): Linear(in_features=64, out_features=16, bias=True)
        (batchnorm0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation0): PReLU(num_parameters=1)
        (dropout0): Dropout(p=0.1, inplace=False)
        (dense1): Linear(in_features=16, out_features=8, bias=True)
        (batchnorm1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation1): PReLU(num_parameters=1)
        (dropout1): D

GPU is available, transfer model to GPU.


In [9]:
print(scores)

[0.5632968350635827, 0.5985434350446499, 0.5694146603136537, 0.5993461866144962]
